# The JuMP ecosystem for mathematical optimization: Introduction

This notebook is part of a workshop at [JuliaCon 2018](http://juliacon.org/2018/) and is based on materials and notebooks from various sources including the [JuliaOpt notebooks](https://github.com/JuliaOpt/juliaopt-notebooks), the [2018 ISCO Spring School](https://github.com/joehuchette/ISCO-spring-school) and the [second annual JuMP-dev workshop](http://www.juliaopt.org/meetings/bordeaux2018/).

## Disclaimer
This notebook is only working under the versions:

- JuMP 0.19 (unreleased, but currently in master)

- MathOptInterface 0.4.1

- GLPK 0.6.0

## Airline Network Revenue Management

<img style="max-width:100%; width:500px; height:auto" src="http://i.imgur.com/jeGwWET.png">

In the airline network revenue management problem we are trying to decide how many tickets for each origin-destination (O-D) pair to sell at each price level. The goal is to maximize revenue, and we cannot sell more tickets than there is demand for, or space on the planes for.

### Three Flight Problem

We'll start with a toy problem that has three origin-destination pairs, with two price classes for each pair. The three origin-destination pairs are BOS-MDW, MDW-SFO, or BOS-SFO via MDW. BOS stands for Boston, MDW is Chicago-Midway, and SFO is San Francisco. Each O-D pair has a "regular" and "discount" fare class. The data we will use is summarized as follows:

```
PLANE CAPACITY: 166

BOS-MDW
        Regular  Discount
Price:  428      190
Demand: 80       120

BOS-SFO
        Regular  Discount
Price:  642      224
Demand: 75       100

MDW-SFO
        Regular  Discount
Price:  512      190
Demand: 60       110
```

In [5]:
using JuMP, GLPK

nrm = Model(optimizer = GLPK.GLPKOptimizerLP())

@variables(nrm, begin 
    0 <= BOStoMDW_R <= 80
    0 <= BOStoMDW_D <= 120
    0 <= BOStoSFO_R <= 75
    0 <= BOStoSFO_D <= 100
    0 <= MDWtoSFO_R <= 60
    0 <= MDWtoSFO_D <= 110
end)

@objective(nrm, Max, 428BOStoMDW_R + 190BOStoMDW_D +
                     642BOStoSFO_R + 224BOStoSFO_D +
                     512MDWtoSFO_R + 190MDWtoSFO_D)

@constraint(nrm, BOStoMDW_R + BOStoMDW_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)
@constraint(nrm, MDWtoSFO_R + MDWtoSFO_D + 
                 BOStoSFO_R + BOStoSFO_D <= 166)

JuMP.optimize(nrm)

@show JuMP.terminationstatus(nrm) == MOI.Success

@show JuMP.resultvalue(BOStoMDW_R)
@show JuMP.resultvalue(BOStoMDW_D)
@show JuMP.objectivevalue(nrm)

JuMP.terminationstatus(nrm) == MOI.Success = true
JuMP.resultvalue(BOStoMDW_R) = 80.0
JuMP.resultvalue(BOStoMDW_D) = 11.0
JuMP.objectivevalue(nrm) = 

121090.0

121090.0


# A step by step example
Let's see how we translate a simple, 2 variable LP to JuMP code.

$$
\begin{align*}
\max_{x,y} \quad& x + 2y \\
\text{s.t.}\quad& x + y \leq 1 \\
& x, y \geq 0.
\end{align*}
$$

First, we load the JuMP, MathOptInterface (MOI), and GLPK (GNU LP/MIP solver).

In [6]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver

Next, we construct a model object. This is a container for everything in our optimization problem: variables, constraints, solver options, etc.

In [7]:
model = Model(optimizer = GLPK.GLPKOptimizerLP())  # Old syntax: model = Model(solver=GLPK.GLPKOptimizerLP())

A JuMP Model

Next, we define the two decision variables in our optimization problem. We will use the ``@variable`` macro. The first argument is the model object to attach the variable to, and the second specifies the variable name and any bounds.

In [8]:
@variable(model, x >= 0)
@variable(model, y >= 0)

y

In [9]:
model

A JuMP Model

We now add the single constraint of our problem using the ``@constraint`` macro. We write it algebraically, exactly as we see it above.

In [10]:
@constraint(model, x + y <= 1)
model

A JuMP Model

We specify the objective function with the ``@objective`` macro.

In [11]:
@objective(model, Max, x + 2y)

In [7]:
model

A JuMP Model

To solve the optimization problem, call the `optimize` function.

In [12]:
JuMP.optimize(model) # Old syntax: status = JuMP.solve(model)

We can then check the status of the optimization call.

In [13]:
@show JuMP.hasresultvalues(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.primalstatus(model) == MOI.FeasiblePoint
@show JuMP.dualstatus(model) == MOI.FeasiblePoint

JuMP.hasresultvalues(model) = true
JuMP.terminationstatus(model) == MOI.Success = true


true

JuMP.primalstatus(model) == MOI.FeasiblePoint = true
JuMP.dualstatus(model) == MOI.FeasiblePoint = true


Much more details than old `:Optimal, :Unbounded, :Infeasible, :UserLimit, :Error, :NotSolved`

In [14]:
display(typeof(MOI.Success))
display(typeof(MOI.FeasiblePoint))

Enum MathOptInterface.TerminationStatusCode:
Success = 0
AlmostSuccess = 1
InfeasibleNoResult = 2
UnboundedNoResult = 3
InfeasibleOrUnbounded = 4
IterationLimit = 5
TimeLimit = 6
NodeLimit = 7
SolutionLimit = 8
MemoryLimit = 9
ObjectiveLimit = 10
NormLimit = 11
OtherLimit = 12
SlowProgress = 13
NumericalError = 14
InvalidModel = 15
InvalidOption = 16
Interrupted = 17
OtherError = 18

Enum MathOptInterface.ResultStatusCode:
FeasiblePoint = 0
NearlyFeasiblePoint = 1
InfeasiblePoint = 2
InfeasibilityCertificate = 3
NearlyInfeasibilityCertificate = 4
ReductionCertificate = 5
NearlyReductionCertificate = 6
UnknownResultStatus = 7
OtherResultStatus = 8

We can now inspect the solution values and optimal cost.

In [15]:
@show JuMP.resultvalue(x)              # Old syntax: getvalue(x)
@show JuMP.resultvalue(y)              # Old syntax: getvalue(y)
@show JuMP.objectivevalue(model)       # Old syntax: getobjectivevalue(model)

2.0

JuMP.resultvalue(x) = 0.0
JuMP.resultvalue(y) = 1.0
JuMP.objectivevalue(model) = 2.0


I can also "name" constraints for later reference.

In [16]:
model = Model(optimizer = GLPK.GLPKOptimizerLP())
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model,inequality, x + y <= 1)         # <=============== give constraint the name "inequality"
@objective(model, Max, x + 2y)
JuMP.optimize(model)
@show JuMP.hasresultvalues(model)
@show JuMP.terminationstatus(model) == MOI.Success
@show JuMP.primalstatus(model) == MOI.FeasiblePoint
@show JuMP.dualstatus(model) == MOI.FeasiblePoint
@show JuMP.resultdual(inequality)

-2.0

JuMP.hasresultvalues(model) = true
JuMP.terminationstatus(model) == MOI.Success = true
JuMP.primalstatus(model) == MOI.FeasiblePoint = true
JuMP.dualstatus(model) == MOI.FeasiblePoint = true
JuMP.resultdual(inequality) = -2.0


Constraint references can be used to modify problem ([By 0.19 Release](https://github.com/JuliaOpt/JuMP.jl/issues/1183)) and to get duals (see Topics notebook).

## Collections of variables/constraints and summation in JuMP

First, we would like to construct a _collection of variables_ all at once.  This is a very common idiom; for example, you might have a variable named ``x`` that is indexed from 1 to 10:

In [17]:
m = Model()
@variable(m, x[1:10] >= 0)

10-element Array{JuMP.VariableRef,1}:
 x[1] 
 x[2] 
 x[3] 
 x[4] 
 x[5] 
 x[6] 
 x[7] 
 x[8] 
 x[9] 
 x[10]

The index sets are specified inside the ``[...]`` block. You can create multidimensional containers by specifying multiple index sets, separated by commas:

In [3]:
@variable(m, y[1:10,["red","blue"]] <= 1)

2-dimensional JuMPArray{JuMP.VariableRef,2,...} with index sets:
    Dimension 1, 1:10
    Dimension 2, String["red", "blue"]
And data, a 10×2 Array{JuMP.VariableRef,2}:
 y[1,red]   y[1,blue] 
 y[2,red]   y[2,blue] 
 y[3,red]   y[3,blue] 
 y[4,red]   y[4,blue] 
 y[5,red]   y[5,blue] 
 y[6,red]   y[6,blue] 
 y[7,red]   y[7,blue] 
 y[8,red]   y[8,blue] 
 y[9,red]   y[9,blue] 
 y[10,red]  y[10,blue]

For more complicated expressions, you can name the indices for the index sets and use them in the rest of the variable definition:

$$
i \leq z_{ij} \leq u_j \;\;\; \forall i \in \{1,...,10\}, j \in \{i+1, ..., 10\}
$$

In [4]:
u = rand(10)
@variable(m, i <= z[i=1:10,j=(i+1):10] <= u[j])

Dict{Any,JuMP.VariableRef} with 45 entries:
  (8, 10) => z[8,10]
  (3, 6)  => z[3,6]
  (6, 9)  => z[6,9]
  (8, 9)  => z[8,9]
  (1, 10) => z[1,10]
  (4, 5)  => z[4,5]
  (2, 4)  => z[2,4]
  (4, 9)  => z[4,9]
  (1, 2)  => z[1,2]
  (3, 4)  => z[3,4]
  (2, 3)  => z[2,3]
  (2, 6)  => z[2,6]
  (2, 5)  => z[2,5]
  (5, 9)  => z[5,9]
  (5, 10) => z[5,10]
  (1, 4)  => z[1,4]
  (3, 9)  => z[3,9]
  (2, 8)  => z[2,8]
  (5, 8)  => z[5,8]
  (4, 8)  => z[4,8]
  (7, 8)  => z[7,8]
  (7, 9)  => z[7,9]
  (5, 7)  => z[5,7]
  (4, 10) => z[4,10]
  (1, 7)  => z[1,7]
  ⋮       => ⋮

To specify conditions on the indexing, you can add conditionals inside the ``[...]`` block, separated by a semicolon:

In [5]:
@variable(m, w[i=1:10, c=["red","blue"]; iseven(i) || c == "red"] >= 0)

Dict{Any,JuMP.VariableRef} with 15 entries:
  (10, "blue") => w[10,blue]
  (7, "red")   => w[7,red]
  (8, "red")   => w[8,red]
  (10, "red")  => w[10,red]
  (6, "blue")  => w[6,blue]
  (1, "red")   => w[1,red]
  (5, "red")   => w[5,red]
  (2, "blue")  => w[2,blue]
  (6, "red")   => w[6,red]
  (3, "red")   => w[3,red]
  (8, "blue")  => w[8,blue]
  (4, "red")   => w[4,red]
  (4, "blue")  => w[4,blue]
  (2, "red")   => w[2,red]
  (9, "red")   => w[9,red]

Now that we can programatically create arrays of variables, we would like to be able to use them to full-effect in the constraints of our problem. That is, we want a way to express multi-dimensional summations, with conditionals. To do this, we use the ``sum(...)`` construction. The first argument is the ''inner loop'' of the summation, the index sets are specified after a ``for``, and any conditionals are stated following an ``if`` (similar to variable definition, but with a slightly different syntax).

$$ \sum _{i = 1}^{10} x_i \leq 1$$

In [6]:
@constraint(m, sum(x[i] for i in 1:10) <= 1)

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(136))

$$ 
\begin{equation}
\sum_{\substack{i\in\{1,...,10\}\\
                c\in\{"red","blue"\}}}
       coef(c) \cdot y_{ic} = 1
\end{equation}
$$

In [7]:
coef = Dict("red" => 2, "blue" => 3)
@constraint(m, sum(coef[c]*y[i,c] for i in 1:10, c in ["red","blue"]) == 1)

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.EqualTo{Float64}}(137))

$$ 
\begin{equation}
\sum_{i = 1}^{10} \sum_{j = i+1}^{10} 
       i \cdot j \cdot z_{ij} \leq
\sum_{\substack{i\in\{1,...,10\},
                c\in\{"red","blue"\} \\
                \text{s.t. } iseven(i) \text{ or } c = "red"}}
       i^2 \cdot w_{ic} 
\end{equation}
$$

In [8]:
@constraint(m, sum(i*j*z[i,j] for i in 1:10, j in (i+1):10) <=
               sum(i^2*w[i,c] for i in 1:10, c in ["red","blue"] if iseven(i) || c == "red"))

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(138))

Can also do collections of constraints (named or unamed):
    $$ 
\begin{align}
x_i &\leq 0.9 \quad \forall i \in \{1,2,3\} \quad\text{ (large bounds)}\\
x_i &\leq 0.5 \quad \forall i \in \{4,5,6\} 
\end{align}
$$

In [28]:
@constraint(m,largebounds[i=1:3], x[i] <= 0.9)
@constraint(m,[i=4:6], x[i] <= 0.5)

1-dimensional JuMPArray{JuMP.ConstraintRef{JuMP.Model,C} where C,1,...} with index sets:
    Dimension 1, 4:6
And data, a 3-element Array{JuMP.ConstraintRef{JuMP.Model,C} where C,1}:
 JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(14))
 JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}(15))
 JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptIn

# New Container Conventions

`JuMPDict` is replaced by `Base.Dict` and `JuMPArray` was rewritten (inspired by `AxisArrays`). Conventions apply for `@variable`, `@constraint`, `@expression`, `@NLconstraint`, `@NLexpression`, ...

In [2]:
m = Model(optimizer = GLPK.GLPKOptimizerLP())
@variable(m, x[1:5, 1:5])            # Array
set_1 = Base.OneTo(5)
@variable(m, y[set_1, 1:5])          # Array
set_2 = 1:5
@variable(m, z[1:5, set_2])          # JuMPArray
set_3 = [:a, :b, :c]
@variable(m, w[set_2, set_3])        # JuMPArray
@variable(m, t[i=set_2, 1:i])        # Dict
@variable(m, h[i = 1:5; isodd(i)])  # Dict

Dict{Any,JuMP.VariableRef} with 3 entries:
  3 => h[3]
  5 => h[5]
  1 => h[1]

`@variable` chooses the tightest applicable container while remaining **type stable**. 
You can also request a container type (for more details see [Internals](Internals.ipynb#JuMP-Containers) notebook):

In [31]:
m = Model(optimizer = GLPK.GLPKOptimizerLP())
@variable(m, x[1:5, 1:5], container = JuMPArray)
set_1 = 1:5
@variable(m, y[set_1, 1:5], container = Array)
set_2 = 2:3
# @variable(m, z[set_2, 1:5], container = Array)  # => Error instead of fallback to JuMPArray to preserve type stability

2:3

New in 0.19: No more slicing error for JuMPArrays!

In [34]:
m = Model(optimizer = GLPK.GLPKOptimizerLP())
set_1 = [:a, :b, :c]
set_2 = [:x, :y, :z]
@variable(m, x[set_1,set_2])
x[:,:z]

1-dimensional JuMPArray{JuMP.VariableRef,1,...} with index sets:
    Dimension 1, Symbol[:a, :b, :c]
And data, a 3-element Array{JuMP.VariableRef,1}:
 x[a,z]
 x[b,z]
 x[c,z]

# Different Classes of Constraints

In [20]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

MathOptInterface.Utilities

Broadcasted and two sided linear inequalities:

In [21]:
A = [1.0 2.0; 3.0 4.0]
l = [4.0, 5.0]
u = [2.0, 3.0]
m = Model()
@variable(m, x[1:2])
@constraint(m, l .<= A*x .<= u)

2-element Array{JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}}},1}:
 JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}}(1))
 JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.Interval{Float64}}(2))

Quadratic Inequalities:

In [22]:
m = Model()
@variable(m, x[1:2])
@constraint(m, x[1]^2 + x[2] <= 1)         # convex 
@constraint(m, x[1]*x[2] - 1.0 == 0.0)     # and non-convex

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.EqualTo{Float64}}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarQuadraticFunction{Float64},MathOptInterface.EqualTo{Float64}}(2))

Conic constraints including...

Semidefinite constraints:

In [23]:
m = Model()                         # using CSDP; m = Model(optimizer = CSDP.CSDPOptimizer())
@variable(m, y[1:2,1:2], Symmetric)
@constraint(m, y in PSDCone())               
@SDconstraint(m,  [t 1; 1 -w] ⪰ [1 t; t -2])

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.PositiveSemidefiniteConeTriangle}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.PositiveSemidefiniteConeTriangle}(2))

Second order cone constraints:
$$ 
\begin{equation}
\left\| Ax+u \right\|_2 \leq t
\end{equation}
$$

In [24]:
m = Model()
@variable(m, x[1:2])
@constraint(m, [t;A*x+u] in SecondOrderCone())

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.SecondOrderCone}(1))

Second order cone constraints:
$$ 
\begin{equation}
\left\| Ax+u \right\|_2 \leq t \cdot w,\quad w\geq 0
\end{equation}
$$

In [25]:
m = Model()
@variable(m, x[1:2])
@constraint(m, [t;w;A*x+u] in RotatedSecondOrderCone())

JuMP.ConstraintRef{JuMP.Model,MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.RotatedSecondOrderCone}}(A JuMP Model, MathOptInterface.ConstraintIndex{MathOptInterface.VectorAffineFunction{Float64},MathOptInterface.RotatedSecondOrderCone}(1))